In [2]:
pip install torch transformers pandas openpyxl xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.4 MB/s eta 0:00:00


In [6]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer

# Load the CodeT5+ model and tokenizer
checkpoint = "Salesforce/codet5p-110m-embedding"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to(device)

# Load the Excel file from the specified path
df = pd.read_excel("/content/PYTHON_AST.xlsx")
print("Data loaded successfully:")
print(df.head())  # Display the first few rows of the DataFrame

# Function to encode data in batches
def encode_data_batch(df, model, tokenizer, device, start_row, end_row):
    embeddings = []
    for i in range(start_row, end_row):
        code = df['AST_PYTHON'][i]  # Use the 'java' column
        inputs = tokenizer.encode(code, return_tensors="pt", max_length=512, truncation=True).to(device)
        with torch.no_grad():
            embedding = model(inputs)[0].cpu().numpy()  # Get fixed-size embeddings
        embeddings.append(embedding)
    return embeddings

# Process the data in batches
batch_size = 300
all_embeddings = []

start_row = 0
while start_row < len(df):
    end_row = min(start_row + batch_size, len(df))
    embeddings_batch = encode_data_batch(df, model, tokenizer, device, start_row, end_row)
    all_embeddings.extend(embeddings_batch)
    start_row = end_row

# Convert to numpy array and reshape if necessary
embeddings_array = np.vstack(all_embeddings)

# Create DataFrame from embeddings
embedding_columns = [f'python_AST_embedding_{i}' for i in range(embeddings_array.shape[1])]
embeddings_df = pd.DataFrame(embeddings_array, columns=embedding_columns)

# Save embeddings to Excel with a neatly formatted sheet
output_path = "/content/python_AST_embeddings.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    embeddings_df.to_excel(writer, index=False, sheet_name="Embeddings")
    workbook = writer.book
    worksheet = writer.sheets["Embeddings"]

    # Set column widths for neat formatting
    for i, column in enumerate(embeddings_df.columns):
        max_len = max(embeddings_df[column].astype(str).map(len).max(), len(column))
        worksheet.set_column(i, i, max_len + 2)  # Adjust the padding as needed

print(f"Embeddings have been saved to {output_path}")

Data loaded successfully:
                                          AST_PYTHON
0  (module (function_definition name: (identifier...
1  (module (function_definition name: (identifier...
2  (module (function_definition name: (identifier...
3  (module (function_definition name: (identifier...
4  (module (function_definition name: (identifier...
Embeddings have been saved to /content/python_AST_embeddings.xlsx


In [7]:
# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap2_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap2_embeddings.xlsx


In [9]:
# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=4, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap4_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap4_embeddings.xlsx


In [10]:
# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=5, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap5_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap5_embeddings.xlsx


In [11]:
# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=6, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5', 'UMAP_6'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap6_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap6_embeddings.xlsx


In [12]:
# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=7, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap7_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap7_embeddings.xlsx


In [21]:
# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=8, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7', 'UMAP_8'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap8_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap8_embeddings.xlsx


In [23]:

# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=9, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7','UMAP_8','UMAP_9'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap9_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap9_embeddings.xlsx


In [19]:

# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=10, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7','UMAP_8','UMAP_9','UMAP_10'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap10_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap10_embeddings.xlsx


In [25]:

# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=11, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7','UMAP_8','UMAP_9','UMAP_10','UMAP_11'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap11_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap11_embeddings.xlsx


In [26]:

# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=12, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7','UMAP_8','UMAP_9','UMAP_10','UMAP_11','UMAP_12'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap12_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap12_embeddings.xlsx


In [27]:

# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=13, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7','UMAP_8','UMAP_9','UMAP_10','UMAP_11','UMAP_12','UMAP_13'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap13_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap13_embeddings.xlsx


In [28]:

# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=14, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7','UMAP_8','UMAP_9','UMAP_10','UMAP_11','UMAP_12','UMAP_13','UMAP_14'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap14_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap14_embeddings.xlsx


In [29]:

# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/python_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=15, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7','UMAP_8','UMAP_9','UMAP_10','UMAP_11','UMAP_12','UMAP_13','UMAP_14','UMAP_15'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/Python_umap15_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/Python_umap15_embeddings.xlsx


In [13]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap2_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 8


In [14]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap3_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 10


In [15]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap4_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 8


In [16]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap5_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 7


In [17]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap6_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 8


In [18]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap7_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 10


In [22]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap8_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 12


In [24]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap9_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 10


In [20]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap10_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 9


In [30]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap11_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 7


In [31]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap12_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 10


In [32]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap13_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 10


In [33]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap14_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 9


In [34]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap15_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 9


In [36]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap8_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Adjust 'eps' and 'min_samples' as needed
cluster_labels = dbscan.fit_predict(embeddings)

# Add cluster labels to the DataFrame
df['Cluster_Label'] = cluster_labels

# Save the DataFrame with cluster labels to a new Excel file
output_file_path = "/content/pythion_ast_row_cluster_embeddings.xlsx"
df.to_excel(output_file_path, index=False)

print(f"Clustered data saved to: {output_file_path}")

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
print(f"Number of clusters formed: {num_clusters}")

Clustered data saved to: /content/row_cluster_embeddings.xlsx
Number of clusters formed: 12


In [45]:
import pandas as pd

# Load the dataset
file_path = '/content/pythion_ast_row_cluster_embeddings.xlsx'
df = pd.read_excel(file_path)

# Check if 'Cluster_Label' exists in the DataFrame
if 'Cluster_Label' in df.columns:
    # Count distinct elements and their occurrences
    distinct_counts = df['Cluster_Label'].value_counts()
    print(distinct_counts)
else:
    print("Column 'Cluster_Label' does not exist in the dataset.")

Cluster_Label
1     19
6     16
11    15
3     15
4     14
5     14
2     13
7     12
8     12
0     10
9      5
10     5
Name: count, dtype: int64


In [41]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap8_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Adjust 'eps' and 'min_samples' as needed
cluster_labels = dbscan.fit_predict(embeddings)

# Replace noise label (-1) with a new cluster ID if there are clusters with fewer than 5 samples
unique_labels, counts = np.unique(cluster_labels, return_counts=True)
for label, count in zip(unique_labels, counts):
    if count < 5:  # Mark clusters with fewer than 5 samples as noise
        cluster_labels[cluster_labels == label] = -1

# Update cluster labels to start from 0
unique_labels = set(cluster_labels)
if -1 in unique_labels:
    unique_labels.remove(-1)
    cluster_labels = np.where(cluster_labels == -1, max(unique_labels) + 1, cluster_labels)

# Add cluster labels to the DataFrame
df['Cluster_Label'] = cluster_labels

# Save the DataFrame with cluster labels to a new Excel file
output_file_path = "/content/pythion_ast_row_cluster_embeddings.xlsx"
df.to_excel(output_file_path, index=False)

print(f"Clustered data saved to: {output_file_path}")

# Find the number of clusters formed
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
print(f"Number of clusters formed: {num_clusters}")


Clustered data saved to: /content/pythion_ast_row_cluster_embeddings.xlsx
Number of clusters formed: 12


In [44]:
import pandas as pd

# Load the dataset
file_path = '/content/row_cluster_embeddings.xlsx'
df = pd.read_excel(file_path)

# Check if 'Cluster_Label' exists in the DataFrame
if 'Cluster_Label' in df.columns:
    # Count distinct elements and their occurrences
    distinct_counts = df['Cluster_Label'].value_counts()
    print(distinct_counts)
else:
    print("Column 'Cluster_Label' does not exist in the dataset.")

Cluster_Label
 1     19
 6     16
 3     15
 4     14
 5     14
 2     13
 7     12
 8     12
-1     11
 0     10
 9      5
 10     5
 11     4
Name: count, dtype: int64


In [46]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap2_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [47]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap3_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 7


In [49]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap4_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [50]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap5_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [51]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap6_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [52]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap7_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [53]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap8_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 7


In [54]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap9_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [55]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap10_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [56]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap11_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [57]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap12_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [58]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap13_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [59]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap14_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [60]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/Python_umap15_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4
